In [67]:
import pandas as pd
import numpy as np
import os, glob
import seaborn as sns

import matplotlib.pyplot as plt

In [103]:
folder_path = "../data/"
processed_path = "../data_processed/"
blasted_path = os.path.join(processed_path, "blasted")

In [56]:
train_values = pd.read_csv(folder_path+"train_values.csv")
test_values = pd.read_csv(folder_path+"test_values.csv")

In [104]:
train_files = glob.glob(blasted_path+"/blasted_train*")
test_files = glob.glob(blasted_path+"/blasted_test*")
print(len(train_files))
print(len(test_files))

100
100


In [16]:
cols = ['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 
        'gapopen', 'sstart', 'send', 'evalue', 'staxids', 
        'sscinames', 'sblastnames', 'stitle']

eval_cutoff = 0.01

each `sseqid` correspond to a unique `stitle`, so `stitle` is a bit redundent.
But we can parse stitle to get a higher level info

In [39]:
df = pd.read_table(train_files[0], names=cols)
df.head()

,qseqid,sseqid,pident,length,mismatch,gapopen,sstart,send,evalue,staxids,sscinames,sblastnames,stitle
0,9OON3,MN996874.1,99.960,2523,0,1,4732,2211,0.000000e+00,NaN,NaN,NaN,"Cloning vector sh-AFP-PPP2CA, complete sequence"
1,9OON3,MN996874.1,99.666,599,2,0,809,1407,0.000000e+00,NaN,NaN,NaN,"Cloning vector sh-AFP-PPP2CA, complete sequence"
2,9OON3,MN996874.1,99.259,135,0,1,1919,1785,1.560000e-58,NaN,NaN,NaN,"Cloning vector sh-AFP-PPP2CA, complete sequence"
3,Y9E43,MN996874.1,100.000,2457,0,0,4732,2276,0.000000e+00,NaN,NaN,NaN,"Cloning vector sh-AFP-PPP2CA, complete sequence"
4,Y9E43,MN996874.1,99.666,599,2,0,809,1407,0.000000e+00,NaN,NaN,NaN,"Cloning vector sh-AFP-PPP2CA, complete sequence"


In [47]:
# for each query seqid, take the first one (because it is the best alignment result)
# remove record with evalue > cutoff (0.01), because it might not be reliable
df_unique = df.groupby('qseqid').first().reset_index()
print("before filtering:", df_unique.shape)
df_unique = df_unique[df_unique.evalue<eval_cutoff]
print("after filtering:", df_unique.shape)

before filtering: (631, 13)
after filtering: (631, 13)


In [34]:
print(df_unique.stitle.unique())

['Human ORFeome Gateway entry vector pENTR223-BTF3, complete sequence'
 'PREDICTED: Hylobates moloch Sec23 homolog A, COPII coat complex component (SEC23A), transcript variant X2, mRNA'
 'Cloning vector PJAC98, complete sequence'
 'Cloning vector p172, complete sequence'
 'Synthetic construct Homo sapiens clone IMAGE:100061543, MGC:190020 general transcription factor IIE, polypeptide 1, alpha 56kDa (GTF2E1) mRNA, encodes complete protein'
 'Human ORFeome Gateway entry vector pENTR223-CSNK2A1, complete sequence'
 'Gateway binary vector pGWB788 DNA, complete sequence'
 'Human ORFeome Gateway entry vector pENTR223-ARF6, complete sequence'
 'Reverse genetics vector pC-N, complete sequence'
 'Cloning vector pTARBAC6, complete sequence'
 'Cloning vector pcDNA3.1_+, complete sequence'
 'Synthetic construct Homo sapiens clone ccsbBroadEn_14136 SAR1B gene, encodes complete protein'
 'Human ORFeome Gateway entry vector pENTR223-BTF3L4, complete sequence'
 'Expression vector pKK44, complete seque

## Aggregate all the blasted data

In [105]:
output_folder = os.path.join(processed_path, "blasted_cleaned")

### train data

In [98]:
df_total = pd.DataFrame()
for fname in train_files:
    df = pd.read_table(fname, names=cols)
    df_unique = df.groupby('qseqid').first().reset_index()
    df_total = df_total.append(df_unique)
df_total = df_total.drop(['staxids', 'sscinames', 'sblastnames'], axis=1)

df_total["blast_ft_eng_1"] = df_total.sseqid.apply(lambda x: x.split("|")[0] if len(x.split("|")) > 1 else "NA")
#df_total["eng_ft_blast_2"] = df_total.sseqid.apply(lambda x: x.split("|")[1] if len(x.split("|")) > 1 else x)

print(df_total.shape)
df_total.to_csv(os.path.join(output_folder, "train_blast_labels.csv"), index=False)

(62919, 11)


In [99]:
train_values_blasted = train_values.merge(df_total, left_on='sequence_id', right_on='qseqid', how='left')
train_values_blasted['blast_aligned_fraction'] = train_values_blasted.apply(lambda x: x['length']/len(x['sequence']), axis=1)

print(train_values_blasted.shape)
train_values_blasted.to_csv(os.path.join(output_folder, "train_values_blasted.csv"), index=False)

(63017, 53)


### test data

In [106]:
df_total = pd.DataFrame()
for fname in test_files:
    df = pd.read_table(fname, names=cols)
    df_unique = df.groupby('qseqid').first().reset_index()
    df_total = df_total.append(df_unique)
df_total = df_total.drop(['staxids', 'sscinames', 'sblastnames'], axis=1)

df_total["blast_ft_eng_1"] = df_total.sseqid.apply(lambda x: x.split("|")[0] if len(x.split("|")) > 1 else "NA")
#df_total["eng_ft_blast_2"] = df_total.sseqid.apply(lambda x: x.split("|")[1] if len(x.split("|")) > 1 else x)

print(df_total.shape)
df_total.to_csv(os.path.join(output_folder, "test_blast_labels.csv"), index=False)

(18782, 11)


In [108]:
test_values_blasted = test_values.merge(df_total, left_on='sequence_id', right_on='qseqid', how='left')
test_values_blasted['blast_aligned_fraction'] = test_values_blasted.apply(lambda x: x['length']/len(x['sequence']), axis=1)

print(test_values_blasted.shape)
test_values_blasted.to_csv(os.path.join(output_folder, "test_values_blasted.csv"), index=False)

(18816, 53)
